### Imports

In [1]:
from mpl_toolkits import mplot3d
from sklearn.metrics import silhouette_score
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from utils.utils import *

import hdbscan
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import sys

%load_ext autotime

/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/li

time: 189 µs (started: 2023-05-24 10:50:56 -05:00)


### Parameters

In [2]:
import json
  
# Opening JSON file
file = open("parameters.json")
params = json.load(file)

dir = os.path.dirname(os.getcwd())
print(dir)

method = params["method"][params["method_idx"]]
print("Method:\t\t", method)

group = params["group"][params["group_idx"]]
print("Group:\t\t", group)

subgroups = [str(k + 1) for k in range(group[1])]
print("Subgroup:\t", subgroups)

dimensions = params["dimensions"]
dimension = dimensions[0]
print("Dimensions:\t", dimensions)

/home/ealvarez/Project/GNN_Filter
Method:		 dgi
Group:		 ['WT', 5]
Subgroup:	 ['1', '2', '3', '4', '5']
Dimensions:	 [3]
time: 3.62 ms (started: 2023-05-24 10:50:56 -05:00)


### Get common subgraphs

In [3]:
graphs = []
for item in tqdm(subgroups):
    weighted_edges = pd.read_csv("{}/output_preprocessing/edges/{}_edges_{}.csv".format(dir, group[0], item))
    G = nx.from_pandas_edgelist(weighted_edges, "source", "target", edge_attr="weight")
    graph_detail(G)
    graphs.append(G)

 20%|██        | 1/5 [00:02<00:11,  2.96s/it]

Num. nodes: 6234
Num. edges: 1243057



 40%|████      | 2/5 [00:05<00:07,  2.49s/it]

Num. nodes: 6231
Num. edges: 939038



 60%|██████    | 3/5 [00:10<00:07,  3.72s/it]

Num. nodes: 6233
Num. edges: 2252847



 80%|████████  | 4/5 [00:19<00:05,  5.91s/it]

Num. nodes: 6226
Num. edges: 3728175



100%|██████████| 5/5 [00:29<00:00,  5.96s/it]

Num. nodes: 6226
Num. edges: 4071713

time: 29.8 s (started: 2023-05-24 10:50:56 -05:00)


In [4]:
H = get_subgraphs(graphs)
graph_detail(H)

100%|██████████| 4/4 [00:37<00:00,  9.33s/it]


Num. nodes: 4202
Num. edges: 135076

time: 43.5 s (started: 2023-05-24 10:51:26 -05:00)


In [5]:
df_edge_embeddings_join_filter_count = pd.DataFrame(H.edges())
df_edge_embeddings_join_filter_count.columns = ["source", "target"]
df_edge_embeddings_join_filter_count

,source,target
0,243.0404,267.1048
1,243.0404,401.2187
2,243.0404,661.3471
3,243.0404,267.0965
4,243.0404,416.1414
...,...,...
135071,579.4245,579.4980
135072,663.3295,663.3449
135073,226.0573,226.0718
135074,412.8122,412.8271


time: 106 ms (started: 2023-05-24 10:52:09 -05:00)


In [6]:
# Get weight
df_edge_embeddings_join_filter_count_weight = df_edge_embeddings_join_filter_count.copy()
s = []
t = []
for row in df_edge_embeddings_join_filter_count_weight.itertuples():
    if row[1] > row[2]:
        s.append(row[2])
        t.append(row[1])
    else:
        s.append(row[1])
        t.append(row[2])
df_edge_embeddings_join_filter_count_weight["source"] = s
df_edge_embeddings_join_filter_count_weight["target"] = t

# df_edge_embeddings_join_filter_count_weight = df_edge_embeddings_join_filter_count.copy()
df_edge_embeddings_join_filter_count_weight.sort_values(["source", "target"], ascending=True, inplace=True)
df_edge_embeddings_join_filter_count_weight["idx"] = df_edge_embeddings_join_filter_count_weight["source"].astype(str) + "-" + df_edge_embeddings_join_filter_count_weight["target"].astype(str)
list_aux = df_edge_embeddings_join_filter_count_weight.iloc[:, -1].values

for i in tqdm(subgroups):
    df_edges = pd.read_csv("{}/output_preprocessing/edges/{}_edges_{}.csv".format(dir, group[0], i))
    df_edges.sort_values(["source", "target"], ascending=True, inplace=True)
    df_edges["idx"] = df_edges["source"].astype(str) + "-" + df_edges["target"].astype(str)

    """ list_temp = []
    # for j in tqdm(range(len(df_edges_filter_count_weight))):
    for row in tqdm(df_edge_embeddings_join_filter_count_weight.itertuples()):
        # weight = 1 # df_edges[(df_edges["source"] == row[1]) & (df_edges["target"] == row[2])].iloc[0, -1]
        weight = df_edges[df_edges["idx"] == "{}-{}".format(row[1], row[2])].iloc[0, -2]
        list_temp.append(weight) """
    
    filter = df_edges["idx"].isin(list_aux)
    temp = df_edges[filter]
    list_temp = temp.iloc[:, -2].values
    df_edge_embeddings_join_filter_count_weight["subgroup{}".format(i)] = list_temp
df_edge_embeddings_join_filter_count_weight


100%|██████████| 5/5 [00:17<00:00,  3.55s/it]


,source,target,idx,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5
133290,59.0137,59.0291,59.0137-59.0291,0.954735,0.928018,0.841093,0.601605,0.970185
131497,59.0137,349.2758,59.0137-349.2758,0.796859,0.788310,0.560861,0.551982,0.807207
110528,59.0137,350.2793,59.0137-350.2793,0.807013,0.779306,0.576143,0.563560,0.808223
132969,59.0137,350.3033,59.0137-350.3033,0.854806,0.751778,0.539172,0.650983,0.883973
129629,59.0137,389.2739,59.0137-389.2739,0.823682,0.886260,0.723124,0.515230,0.917952
...,...,...,...,...,...,...,...,...
130790,980.7146,981.7177,980.7146-981.7177,0.743341,0.954130,0.878971,0.816034,0.817595
108711,981.5305,981.7177,981.5305-981.7177,0.571209,0.850899,0.670445,0.544705,0.731794
131593,981.7177,996.7096,981.7177-996.7096,0.557537,0.784976,0.563671,0.732291,0.554250
134700,985.5091,985.5490,985.5091-985.549,0.798737,0.577365,0.723628,0.875572,0.644783


time: 18 s (started: 2023-05-24 10:52:09 -05:00)


In [7]:
# Dispersion (std)
df_edge_embeddings_join_filter_count_weight_std = df_edge_embeddings_join_filter_count_weight.copy()
df_edge_embeddings_join_filter_count_weight_std["std"] = np.std(df_edge_embeddings_join_filter_count_weight_std.iloc[:, -len(subgroups):], axis=1)
df_edge_embeddings_join_filter_count_weight_std

,source,target,idx,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5,std
133290,59.0137,59.0291,59.0137-59.0291,0.954735,0.928018,0.841093,0.601605,0.970185,0.136282
131497,59.0137,349.2758,59.0137-349.2758,0.796859,0.788310,0.560861,0.551982,0.807207,0.118268
110528,59.0137,350.2793,59.0137-350.2793,0.807013,0.779306,0.576143,0.563560,0.808223,0.112406
132969,59.0137,350.3033,59.0137-350.3033,0.854806,0.751778,0.539172,0.650983,0.883973,0.128241
129629,59.0137,389.2739,59.0137-389.2739,0.823682,0.886260,0.723124,0.515230,0.917952,0.145148
...,...,...,...,...,...,...,...,...,...
130790,980.7146,981.7177,980.7146-981.7177,0.743341,0.954130,0.878971,0.816034,0.817595,0.070631
108711,981.5305,981.7177,981.5305-981.7177,0.571209,0.850899,0.670445,0.544705,0.731794,0.111292
131593,981.7177,996.7096,981.7177-996.7096,0.557537,0.784976,0.563671,0.732291,0.554250,0.099503
134700,985.5091,985.5490,985.5091-985.549,0.798737,0.577365,0.723628,0.875572,0.644783,0.106148


time: 56.8 ms (started: 2023-05-24 10:52:28 -05:00)


In [8]:
# Filter by std
df_edge_embeddings_join_filter_count_weight_std = df_edge_embeddings_join_filter_count_weight_std[df_edge_embeddings_join_filter_count_weight_std["std"] <= 0.3]
df_edge_embeddings_join_filter_count_weight_std.to_csv("{}/output_greedy/edges_filter_weight_std/{}_edge-filter-weight-std_{}_{}.csv".format(dir, group[0], dimension, "L2"), index=False)
df_edge_embeddings_join_filter_count_weight_std

,source,target,idx,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5,std
133290,59.0137,59.0291,59.0137-59.0291,0.954735,0.928018,0.841093,0.601605,0.970185,0.136282
131497,59.0137,349.2758,59.0137-349.2758,0.796859,0.788310,0.560861,0.551982,0.807207,0.118268
110528,59.0137,350.2793,59.0137-350.2793,0.807013,0.779306,0.576143,0.563560,0.808223,0.112406
132969,59.0137,350.3033,59.0137-350.3033,0.854806,0.751778,0.539172,0.650983,0.883973,0.128241
129629,59.0137,389.2739,59.0137-389.2739,0.823682,0.886260,0.723124,0.515230,0.917952,0.145148
...,...,...,...,...,...,...,...,...,...
130790,980.7146,981.7177,980.7146-981.7177,0.743341,0.954130,0.878971,0.816034,0.817595,0.070631
108711,981.5305,981.7177,981.5305-981.7177,0.571209,0.850899,0.670445,0.544705,0.731794,0.111292
131593,981.7177,996.7096,981.7177-996.7096,0.557537,0.784976,0.563671,0.732291,0.554250,0.099503
134700,985.5091,985.5490,985.5091-985.549,0.798737,0.577365,0.723628,0.875572,0.644783,0.106148


time: 1.25 s (started: 2023-05-24 10:52:28 -05:00)


In [9]:
df_edge_embeddings_join_filter_count_weight_std = pd.read_csv("{}/output_greedy/edges_filter_weight_std/{}_edge-filter-weight-std_{}_{}.csv".format(dir, group[0], dimension, "L2"))
df_edge_embeddings_join_filter_count_weight_std

,source,target,idx,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5,std
0,59.0137,59.0291,59.0137-59.0291,0.954735,0.928018,0.841093,0.601605,0.970185,0.136282
1,59.0137,349.2758,59.0137-349.2758,0.796859,0.788310,0.560861,0.551982,0.807207,0.118268
2,59.0137,350.2793,59.0137-350.2793,0.807013,0.779306,0.576143,0.563560,0.808223,0.112406
3,59.0137,350.3033,59.0137-350.3033,0.854806,0.751778,0.539172,0.650983,0.883973,0.128241
4,59.0137,389.2739,59.0137-389.2739,0.823682,0.886260,0.723124,0.515230,0.917952,0.145148
...,...,...,...,...,...,...,...,...,...
131535,980.7146,981.7177,980.7146-981.7177,0.743341,0.954130,0.878971,0.816034,0.817595,0.070631
131536,981.5305,981.7177,981.5305-981.7177,0.571209,0.850899,0.670445,0.544705,0.731794,0.111292
131537,981.7177,996.7096,981.7177-996.7096,0.557537,0.784976,0.563671,0.732291,0.554250,0.099503
131538,985.5091,985.5490,985.5091-985.549,0.798737,0.577365,0.723628,0.875572,0.644783,0.106148


time: 211 ms (started: 2023-05-24 10:52:29 -05:00)


In [10]:
# Average weight
df_edge_embeddings_join_filter_count_weight_std_avg = df_edge_embeddings_join_filter_count_weight_std.copy()
df_edge_embeddings_join_filter_count_weight_std_avg["weight"] = df_edge_embeddings_join_filter_count_weight_std_avg.iloc[:, -(len(subgroups) + 1):-1].mean(axis=1)
df_edge_embeddings_join_filter_count_weight_std_avg

,source,target,idx,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5,std,weight
0,59.0137,59.0291,59.0137-59.0291,0.954735,0.928018,0.841093,0.601605,0.970185,0.136282,0.859127
1,59.0137,349.2758,59.0137-349.2758,0.796859,0.788310,0.560861,0.551982,0.807207,0.118268,0.701044
2,59.0137,350.2793,59.0137-350.2793,0.807013,0.779306,0.576143,0.563560,0.808223,0.112406,0.706849
3,59.0137,350.3033,59.0137-350.3033,0.854806,0.751778,0.539172,0.650983,0.883973,0.128241,0.736142
4,59.0137,389.2739,59.0137-389.2739,0.823682,0.886260,0.723124,0.515230,0.917952,0.145148,0.773249
...,...,...,...,...,...,...,...,...,...,...
131535,980.7146,981.7177,980.7146-981.7177,0.743341,0.954130,0.878971,0.816034,0.817595,0.070631,0.842014
131536,981.5305,981.7177,981.5305-981.7177,0.571209,0.850899,0.670445,0.544705,0.731794,0.111292,0.673811
131537,981.7177,996.7096,981.7177-996.7096,0.557537,0.784976,0.563671,0.732291,0.554250,0.099503,0.638545
131538,985.5091,985.5490,985.5091-985.549,0.798737,0.577365,0.723628,0.875572,0.644783,0.106148,0.724017


time: 39.4 ms (started: 2023-05-24 10:52:29 -05:00)


In [11]:
df_edge_embeddings_join_filter_count_weight_std_avg = df_edge_embeddings_join_filter_count_weight_std_avg.iloc[:, [0, 1, -1]]
df_edge_embeddings_join_filter_count_weight_std_avg.to_csv("{}/output_greedy/edges_filter_weight_std_avg/{}_edge-filter-weight-std-avg_{}_{}.csv".format(dir, group[0], dimension, "L2"), index=False)
df_edge_embeddings_join_filter_count_weight_std_avg

,source,target,weight
0,59.0137,59.0291,0.859127
1,59.0137,349.2758,0.701044
2,59.0137,350.2793,0.706849
3,59.0137,350.3033,0.736142
4,59.0137,389.2739,0.773249
...,...,...,...
131535,980.7146,981.7177,0.842014
131536,981.5305,981.7177,0.673811
131537,981.7177,996.7096,0.638545
131538,985.5091,985.5490,0.724017


time: 354 ms (started: 2023-05-24 10:52:29 -05:00)


In [12]:
df_edge_embeddings_join_filter_count_weight_std_avg = pd.read_csv("{}/output_greedy/edges_filter_weight_std_avg/{}_edge-filter-weight-std-avg_{}_{}.csv".format(dir, group[0], dimension, "L2"))
df_edge_embeddings_join_filter_count_weight_std_avg

,source,target,weight
0,59.0137,59.0291,0.859127
1,59.0137,349.2758,0.701044
2,59.0137,350.2793,0.706849
3,59.0137,350.3033,0.736142
4,59.0137,389.2739,0.773249
...,...,...,...
131535,980.7146,981.7177,0.842014
131536,981.5305,981.7177,0.673811
131537,981.7177,996.7096,0.638545
131538,985.5091,985.5490,0.724017


time: 50.7 ms (started: 2023-05-24 10:52:30 -05:00)


In [13]:
G = nx.from_pandas_edgelist(df_edge_embeddings_join_filter_count_weight_std_avg, "source", "target", edge_attr=["weight"])
graph_detail(G)

Num. nodes: 4181
Num. edges: 131540

time: 261 ms (started: 2023-05-24 10:52:30 -05:00)


### Clustering

In [17]:
import pandas as pd
import researchpy as rp

time: 22.2 ms (started: 2023-05-24 11:22:27 -05:00)


In [18]:
df = pd.read_csv("https://raw.githubusercontent.com/researchpy/Data-sets/master/difficile.csv")
df.drop('person', axis= 1, inplace= True)

# Recoding value from numeric to string
df['dose'].replace({1: 'placebo', 2: 'low', 3: 'high'}, inplace= True)

df

,dose,libido
0,placebo,3
1,placebo,2
2,placebo,1
3,placebo,1
4,placebo,4
5,low,5
6,low,2
7,low,4
8,low,2
9,low,3


time: 379 ms (started: 2023-05-24 11:23:14 -05:00)
